# Creating LIB, CORPUS, and VOCAB tables

Charlie Perez (cwp5xyj)

A note that the process of clipping the text files and creating the LIB table will be largely done by hand - I really want to preserve things like the Table of Contents for later work (given Martin's writing style, the exact POV of the chapter is important).

In [53]:
import numpy as np
import pandas as pd

from glob import glob
import re
import nltk
import plotly_express as px
import configparser

import os

In [54]:
from collections import defaultdict

In [84]:
# Think I will have to remove Fire and Blood - it wouldn't match well anyways, and would be much more of a novelty

metadata = [
    {
        'ID': 1,
        'file': 'agot.txt',
        'title': 'A Game of Thrones',
        'clip_range': (212, 14145),
        'chap_regex': r'^[A-Z ]+$',
        'TOC_range': (20, 165)
    },
    {
        'ID': 2,
        'file': 'acok.txt',
        'title': 'A Clash of Kings',
        'clip_range': (382, 16150),
        'chap_regex': r'^[A-Z ]+$',
        'TOC_range': (55, 334)
    },
    {
        'ID': 3,
        'file': 'asos.txt',
        'title': 'A Storm of Swords',
        'clip_range': (451, 20253),
        'chap_regex': r'^[A-Z ]+$',
        'TOC_range': (59, 386)
    },
    {
        'ID': 4,
        'file': 'affc.txt',
        'title': 'A Feast for Crows',
        'clip_range': (343, 13963),
        'chap_regex': r"^[A-Z’‘ʼ' ]+$",
        'TOC_range': (87, 271)
    },
    {
        'ID': 5,
        'file': 'adwd.txt',
        'title': 'A Dance with Dragons',
        'clip_range': (344, 18874),
        'chap_regex': r"^[A-Z’‘ʼ' ]+$",
        'TOC_range': (104, 250)
    }
]

#### Create LIB table

With formatted table of contents

In [85]:
data_dir = f'{os.getcwd()}/data'
data_dir

'/sfs/gpfs/tardis/home/cwp5xyj/Documents/MSDS/DS5001/final-project/data'

In [86]:
LIB = pd.DataFrame(metadata)
LIB.file = data_dir + '/' + LIB.file
LIB.set_index('ID', inplace=True)
LIB

,file,title,clip_range,chap_regex,TOC_range
ID,,,,,
1,/sfs/gpfs/tardis/home/cwp5xyj/Documents/MSDS/D...,A Game of Thrones,"(212, 14145)",^[A-Z ]+$,"(20, 165)"
2,/sfs/gpfs/tardis/home/cwp5xyj/Documents/MSDS/D...,A Clash of Kings,"(382, 16150)",^[A-Z ]+$,"(55, 334)"
3,/sfs/gpfs/tardis/home/cwp5xyj/Documents/MSDS/D...,A Storm of Swords,"(451, 20253)",^[A-Z ]+$,"(59, 386)"
4,/sfs/gpfs/tardis/home/cwp5xyj/Documents/MSDS/D...,A Feast for Crows,"(343, 13963)",^[A-Z’‘ʼ' ]+$,"(87, 271)"
5,/sfs/gpfs/tardis/home/cwp5xyj/Documents/MSDS/D...,A Dance with Dragons,"(344, 18874)",^[A-Z’‘ʼ' ]+$,"(104, 250)"


In [87]:
for idx, row in LIB.iterrows():
    src, clip_range, chap_regex, TOC_range = row.file, row.clip_range, row.chap_regex, row.TOC_range
    print(src, clip_range, chap_regex, TOC_range)

/sfs/gpfs/tardis/home/cwp5xyj/Documents/MSDS/DS5001/final-project/data/agot.txt (212, 14145) ^[A-Z ]+$ (20, 165)
/sfs/gpfs/tardis/home/cwp5xyj/Documents/MSDS/DS5001/final-project/data/acok.txt (382, 16150) ^[A-Z ]+$ (55, 334)
/sfs/gpfs/tardis/home/cwp5xyj/Documents/MSDS/DS5001/final-project/data/asos.txt (451, 20253) ^[A-Z ]+$ (59, 386)
/sfs/gpfs/tardis/home/cwp5xyj/Documents/MSDS/DS5001/final-project/data/affc.txt (343, 13963) ^[A-Z’‘ʼ' ]+$ (87, 271)
/sfs/gpfs/tardis/home/cwp5xyj/Documents/MSDS/DS5001/final-project/data/adwd.txt (344, 18874) ^[A-Z’‘ʼ' ]+$ (104, 250)


In [88]:
def read_lines(src):
    text_lines = open(src,'r', encoding= 'utf-8').readlines()
    LINES = pd.DataFrame({'line_str':text_lines})
    LINES.index.name = 'line_id'
    return LINES

def extract_TOC(LINES, TOC_range):
    TOC_lines = LINES.loc[TOC_range[0]:TOC_range[1]].copy()
    contents = []
    chapter_counts = defaultdict(int)
    for i in TOC_lines.line_str.str.strip('\n'):
        if i != '':
            if i != 'Prologue' and i != 'Epilogue':
                chapter_counts[i] += 1
                contents.append(f'{i} {chapter_counts[i]}')
            else:
                contents.append(i)
    return contents

def parse_tokens(LINES, clip_range, chap_regex, book_id):
    LINES = LINES.loc[clip_range[0]:clip_range[1]].copy()
    OHCO = ['chap_num', 'para_num', 'sent_num', 'token_num']
    
    LINES.line_str = LINES.line_str.str.replace(r'\n+', ' ', regex=True).str.strip()
    chap_lines = LINES.line_str.str.match(chap_regex)

    LINES.loc[chap_lines, 'chap_num'] = [i+1 for i in range(LINES.loc[chap_lines].shape[0])]
    LINES.chap_num = LINES.chap_num.ffill()

    LINES = LINES.dropna(subset=['chap_num'])
    LINES = LINES.loc[~chap_lines]
    LINES.chap_num = LINES.chap_num.astype('int')

    CHAPS = LINES.groupby(OHCO[:1])\
        .line_str.apply(lambda x: '\n'.join(x))\
        .to_frame('chap_str')

    CHAPS['chap_str'] = CHAPS.chap_str.str.strip()

    para_pat = r'\n\n+'

    PARAS = CHAPS['chap_str'].str.split(para_pat, expand=True).stack()\
        .to_frame('para_str').sort_index()
    PARAS.index.names = OHCO[:2]

    PARAS['para_str'] = PARAS['para_str'].str.replace(r'\n', ' ', regex=True)
    PARAS['para_str'] = PARAS['para_str'].str.strip()
    PARAS = PARAS[~PARAS['para_str'].str.match(r'^\s*$')]

    SENTS = PARAS.para_str\
                    .apply(lambda x: pd.Series(nltk.sent_tokenize(x), dtype='string'))\
                    .stack()\
                    .to_frame('sent_str')

    SENTS.index.names = OHCO[:3]

    TOKENS = SENTS.sent_str\
                    .apply(lambda x: pd.Series(nltk.pos_tag(nltk.word_tokenize(x))))
    TOKENS = TOKENS.stack().to_frame('pos_tuple')
    TOKENS['pos'] = TOKENS.pos_tuple.apply(lambda x: x[1])
    TOKENS['token_str'] = TOKENS.pos_tuple.apply(lambda x: x[0])
    TOKENS['term_str'] = TOKENS.token_str.str.lower()

    TOKENS.index.names = OHCO[:4]
    
    TOKENS.reset_index(inplace=True)
    TOKENS['book_id'] = book_id
    OHCO = ['book_id', 'chap_num', 'para_num', 'sent_num', 'token_num']
    TOKENS.set_index(OHCO, inplace=True)
    
    punc_pos = ['$', "''", '(', ')', ',', '--', '.', ':', '``']
    TOKENS['term_str'] = TOKENS[~TOKENS.pos.isin(punc_pos)].token_str\
                        .str.replace(r'[\W_]+', '', regex=True).str.lower()  
    return TOKENS
    
    
def parse_corpus(LIB):
    contents = []
    TOKENS = pd.DataFrame()
    for idx, row in LIB.iterrows():
        book_id, src, clip_range, chap_regex, TOC_range = idx, row.file, row.clip_range, row.chap_regex, row.TOC_range
        LINES = read_lines(src)
        ind_contents = extract_TOC(LINES, TOC_range)
        ind_TOKENS = parse_tokens(LINES, clip_range, chap_regex, book_id)
        
        contents.append(ind_contents)
        TOKENS = pd.concat([TOKENS, ind_TOKENS])
    
    return contents, TOKENS

In [89]:
import time

In [90]:
%%time

contents, TOKENS = parse_corpus(LIB)

CPU times: user 1min 44s, sys: 619 ms, total: 1min 45s
Wall time: 1min 45s


In [91]:
TOKENS.head()

pos_tuple  pos token_str  \
book_id chap_num para_num sent_num token_num                                
1       1        0        0        0               (“, IN)   IN         “   
                                   1             (We, PRP)  PRP        We   
                                   2          (should, MD)   MD    should   
                                   3           (start, VB)   VB     start   
                                   4            (back, RB)   RB      back   

                                             term_str  
book_id chap_num para_num sent_num token_num           
1       1        0        0        0                   
                                   1               we  
                                   2           should  
                                   3            start  
                                   4             back

In [92]:
TOKENS.tail()

pos_tuple  pos token_str  \
book_id chap_num para_num sent_num token_num                                  
5       73       140      0        3            (hands, NNS)  NNS     hands   
                                   4                  (,, ,)    ,         ,   
                                   5               (the, DT)   DT       the   
                                   6          (daggers, NNS)  NNS   daggers   
                                   7                  (., .)    .         .   

                                             term_str  
book_id chap_num para_num sent_num token_num           
5       73       140      0        3            hands  
                                   4              NaN  
                                   5              the  
                                   6          daggers  
                                   7              NaN

In [93]:
TOKENS.reset_index()[['book_id','chap_num']]\
    .drop_duplicates()\
    .groupby('book_id').chap_num.count()

book_id
1    73
2    70
3    82
4    46
5    73
Name: chap_num, dtype: int64

In [94]:
print([len(i) for i in contents])

[73, 70, 82, 46, 73]


Need to manually mess with the AFFC and ADWD chapters - gonna be a bit of a pain

Session died. Tears. Not gonna reload everything right now.

#### Create Vocabulary from TOKENS table

In [95]:
CORPUS = TOKENS # forgot it was supposed to be called CORPUS
CORPUS = CORPUS[CORPUS.term_str != ''].copy() # get rid of random punctuation
CORPUS['pos_group'] = CORPUS.pos.str[:2] # also didn't realize this was needed too

VOCAB = CORPUS.term_str.value_counts().to_frame('n')
VOCAB.index.name = 'term_str'
VOCAB['p'] = VOCAB['n'] / VOCAB['n'].sum()
VOCAB['s'] = 1 / VOCAB['p']
VOCAB['i'] = np.log2(VOCAB['s']) 
VOCAB['max_pos'] = CORPUS[['term_str','pos']].value_counts().unstack(fill_value=0).idxmax(1)
VOCAB['max_pos_group'] = CORPUS[['term_str','pos_group']].value_counts().unstack(fill_value=0).idxmax(1)

In [96]:
# add features
LIB['date'] = [1996, 1998, 2000, 2005, 2011]
LIB['chap_labels'] = contents

I understand that I'm short some metadata (I contemplated branching out into other book series, or a different type of ASOIAF books). But I like what I have going on here and am disinclined to change it.

In [97]:
# save these two as they likely won't be changed again

LIB.to_csv('output/LIB.csv', sep='|')
CORPUS.to_csv('output/CORPUS.csv', sep='|')

#### Getting into BOW, TFIDF, and DFIDF

No point keeping these separate. Need DFIDF in the VOCAB table, so may as well do it here.

Chapter is the unit of observation most interesting to me, so that's what we're gonna go with here. TF method is 'sum' and IDF method is 'standard'.

In [98]:
bag = ['book_id', 'chap_num']
BOW = CORPUS.groupby(bag+['term_str']).term_str.count().to_frame('n')
DTCM = BOW.n.unstack(fill_value=0)
N = DTCM.shape[0]
TF = (DTCM.T / DTCM.T.sum()).T
DF = DTCM.astype('bool').sum()
IDF = np.log2(N / DF)
TFIDF = TF * IDF

VOCAB['df'] = DF
VOCAB['dfidf'] = VOCAB.df * np.log2(len(TFIDF)/VOCAB.df)

In [99]:
VOCAB.sort_values(by='dfidf', ascending=False).head(20)

,n,p,s,i,max_pos,max_pos_group,df,dfidf
term_str,,,,,,,,
lannisters,310,0.000176,5693.232258,12.475032,NNP,NN,127,182.572669
whilst,286,0.000162,6170.986014,12.591285,NN,VB,127,182.572669
laughing,183,0.000104,9644.273224,13.235457,VBG,VB,127,182.572669
frowned,155,0.000088,11386.464516,13.475032,VBD,VB,127,182.572669
fist,231,0.000131,7640.268398,12.899408,NN,NN,127,182.572669
didn,275,0.000156,6417.825455,12.647869,VBP,VB,127,182.572669
recall,177,0.000100,9971.197740,13.283551,VB,VB,126,182.572089
sorry,189,0.000107,9338.105820,13.188914,JJ,JJ,126,182.572089
stupid,253,0.000143,6975.897233,12.768163,JJ,JJ,126,182.572089


In [100]:
# now for stemming and stopwords

sw = pd.DataFrame(nltk.corpus.stopwords.words('english'), columns=['term_str'])
sw = sw.reset_index().set_index('term_str')
sw.columns = ['dummy']
sw.dummy = 1

VOCAB['stop'] = VOCAB.index.map(sw.dummy)
VOCAB['stop'] = VOCAB['stop'].fillna(0).astype('int')

In [101]:
from nltk.stem.porter import PorterStemmer
stemmer1 = PorterStemmer()
VOCAB['porter_stem'] = VOCAB.apply(lambda x: stemmer1.stem(x.name), 1)
VOCAB.head()

,n,p,s,i,max_pos,max_pos_group,df,dfidf,stop,porter_stem
term_str,,,,,,,,,,
the,101699,0.057623,17.354173,4.117211,DT,DT,344,0.0,1,the
and,51225,0.029024,34.453919,5.106596,CC,CC,344,0.0,1,and
to,39497,0.022379,44.684457,5.481701,TO,TO,344,0.0,1,to
a,39269,0.022250,44.943900,5.490053,DT,DT,344,0.0,1,a
of,35196,0.019942,50.144960,5.648033,IN,IN,344,0.0,1,of


In [102]:
VOCAB.to_csv('output/VOCAB.csv', sep='|')

In [103]:
# taking a look at the BOW, DCTM, and TFIDF matrices

BOW['tfidf'] = TFIDF.stack()

BOW.head(10)

n     tfidf
book_id chap_num term_str                   
1       1        a             111  0.000000
                 abandoned       2  0.001161
                 about           4  0.000053
                 above           1  0.000120
                 accustomed      1  0.000878
                 acquiescence    1  0.002188
                 across          1  0.000026
                 adjusted        1  0.001409
                 admitted        1  0.000463
                 aemon           2  0.001260

In [104]:
DTCM.head()

term_str          10th  15th  16th  23rd  57th  61st    a  \
book_id chap_num                                            
1       1            0     0     0     0     0     0  111   
        2            0     0     0     0     0     0   63   
        3            0     0     0     0     0     0   41   
        4            0     0     0     0     0     0   78   
        5            0     0     0     0     0     0   86   

term_str          aaaaaaarrreeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee  \
book_id chap_num                                                          
1       1                                                         0       
        2                                                         0       
        3                                                         0       
        4                                                         0       
        5                                                         0       

term_str          aaaaaaoooooooooooooooooooooooo  aaaaahoooooooooooooooooooo  \
book_id chap_num                                                               
1       1                                      0                           0   
        2                                      0                           0   
        3                                      0                           0   
        4                                      0                           0   
        5                                      0                           0   

term_str          ...  zekko  zenith  zhak  zharaq  zigged  zo  zollo  zorse  \
book_id chap_num  ...                                                          
1       1         ...      0       0     0       0       0   0      0      0   
        2         ...      0       0     0       0       0   0      0      0   
        3         ...      0       0     0       0       0   0      0      0   
        4         ...      0       0     0       0       0   0      0      0   
        5         ...      0       0     0       0       0   0      0      0   

term_str          zorses  zzzs  
book_id chap_num                
1       1              0     0  
        2              0     0  
        3              0     0  
        4              0     0  
        5              0     0  

[5 rows x 24235 columns]

It may look like there's an issue with the source material, but there are actually just several different horn sounds in one particular chapter in *A Feast for Crows*.

In [105]:
TFIDF.head()

term_str          10th  15th  16th  23rd  57th  61st    a  \
book_id chap_num                                            
1       1          0.0   0.0   0.0   0.0   0.0   0.0  0.0   
        2          0.0   0.0   0.0   0.0   0.0   0.0  0.0   
        3          0.0   0.0   0.0   0.0   0.0   0.0  0.0   
        4          0.0   0.0   0.0   0.0   0.0   0.0  0.0   
        5          0.0   0.0   0.0   0.0   0.0   0.0  0.0   

term_str          aaaaaaarrreeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee  \
book_id chap_num                                                          
1       1                                                       0.0       
        2                                                       0.0       
        3                                                       0.0       
        4                                                       0.0       
        5                                                       0.0       

term_str          aaaaaaoooooooooooooooooooooooo  aaaaahoooooooooooooooooooo  \
book_id chap_num                                                               
1       1                                    0.0                         0.0   
        2                                    0.0                         0.0   
        3                                    0.0                         0.0   
        4                                    0.0                         0.0   
        5                                    0.0                         0.0   

term_str          ...  zekko  zenith  zhak  zharaq  zigged   zo  zollo  zorse  \
book_id chap_num  ...                                                           
1       1         ...    0.0     0.0   0.0     0.0     0.0  0.0    0.0    0.0   
        2         ...    0.0     0.0   0.0     0.0     0.0  0.0    0.0    0.0   
        3         ...    0.0     0.0   0.0     0.0     0.0  0.0    0.0    0.0   
        4         ...    0.0     0.0   0.0     0.0     0.0  0.0    0.0    0.0   
        5         ...    0.0     0.0   0.0     0.0     0.0  0.0    0.0    0.0   

term_str          zorses  zzzs  
book_id chap_num                
1       1            0.0   0.0  
        2            0.0   0.0  
        3            0.0   0.0  
        4            0.0   0.0  
        5            0.0   0.0  

[5 rows x 24235 columns]

So I am guessing that "reduced and normalized TFIDF_L2" means I have to reduce it to some specific feature size, probably by DFIDF.

I think I want to take out proper nouns (names) but probably keep verbs and adjectives? Is 5000 too many?

In [106]:
VSHORT = VOCAB[VOCAB.max_pos_group.isin(['NN', 'VB', 'JJ']) & ~VOCAB.max_pos.isin(['NNP'])].sort_values('dfidf', ascending=False).head(2000)

TFIDF_2000 = TFIDF[VSHORT.index]

In [107]:
from scipy.linalg import norm

In [108]:
# create TFIDF_L2

TFIDF_L2 = (TFIDF_2000.T / norm(TFIDF_2000, 2, axis=1)).T

a = len(TFIDF_L2)
TFIDF_L2 = TFIDF_L2.dropna()
b = len(TFIDF_L2)
bag_loss = a - b
bag_loss

0

In [109]:
TFIDF_L2

term_str              didn    whilst  laughing      fist   frowned     doors  \
book_id chap_num                                                               
1       1         0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
        2         0.029764  0.000000  0.059528  0.000000  0.029764  0.000000   
        3         0.000000  0.000000  0.000000  0.000000  0.031174  0.000000   
        4         0.000000  0.000000  0.000000  0.000000  0.000000  0.043713   
        5         0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
...                    ...       ...       ...       ...       ...       ...   
5       69        0.000000  0.014151  0.000000  0.000000  0.000000  0.142629   
        70        0.000000  0.048142  0.032094  0.016047  0.000000  0.048524   
        71        0.000000  0.030067  0.000000  0.045100  0.000000  0.000000   
        72        0.000000  0.000000  0.009752  0.019504  0.000000  0.000000   
        73        0.000000  0.052250  0.000000  0.017417  0.000000  0.000000   

term_str             sorry   stupid   leaving    leaves  ...   quicker  \
book_id chap_num                                         ...             
1       1         0.000000  0.00000  0.000000  0.071728  ...  0.000000   
        2         0.000000  0.00000  0.000000  0.000000  ...  0.000000   
        3         0.031421  0.00000  0.000000  0.031421  ...  0.000000   
        4         0.000000  0.00000  0.000000  0.043713  ...  0.000000   
        5         0.000000  0.00000  0.028599  0.000000  ...  0.000000   
...                    ...      ...       ...       ...  ...       ...   
5       69        0.000000  0.00000  0.000000  0.000000  ...  0.061834   
        70        0.032349  0.00000  0.000000  0.000000  ...  0.000000   
        71        0.000000  0.00000  0.000000  0.000000  ...  0.000000   
        72        0.000000  0.00983  0.000000  0.009830  ...  0.000000   
        73        0.035110  0.00000  0.000000  0.000000  ...  0.000000   

term_str          shuddered  knowledge    height  honorable   trimmed  \
book_id chap_num                                                        
1       1          0.000000        0.0  0.000000   0.000000  0.000000   
        2          0.000000        0.0  0.000000   0.000000  0.065029   
        3          0.000000        0.0  0.000000   0.000000  0.000000   
        4          0.000000        0.0  0.000000   0.047377  0.000000   
        5          0.000000        0.0  0.061991   0.000000  0.000000   
...                     ...        ...       ...        ...       ...   
5       69         0.000000        0.0  0.000000   0.000000  0.000000   
        70         0.000000        0.0  0.000000   0.000000  0.035060   
        71         0.000000        0.0  0.000000   0.000000  0.000000   
        72         0.000000        0.0  0.000000   0.000000  0.000000   
        73         0.038053        0.0  0.000000   0.000000  0.000000   

term_str           freedom  pressing  consider  uncomfortable  
book_id chap_num                                               
1       1         0.000000  0.000000  0.000000            0.0  
        2         0.000000  0.000000  0.000000            0.0  
        3         0.000000  0.000000  0.000000            0.0  
        4         0.000000  0.000000  0.000000            0.0  
        5         0.000000  0.061991  0.061991            0.0  
...                    ...       ...       ...            ...  
5       69        0.000000  0.000000  0.000000            0.0  
        70        0.000000  0.000000  0.000000            0.0  
        71        0.000000  0.032845  0.000000            0.0  
        72        0.021307  0.000000  0.000000            0.0  
        73        0.000000  0.038053  0.000000            0.0  

[344 rows x 2000 columns]

In [110]:
BOW.to_csv('output/BOW.csv', sep='|')
DTCM.to_csv('output/DTM.csv', sep='|')
TFIDF.to_csv('output/TFIDF.csv', sep='|')
TFIDF_L2.to_csv('output/TFIDF_L2.csv', sep='|')